In [1]:
import pandas as pd
import os
import optuna

import cv2  # 画像認識するのはcv2
import keras

import numpy as np
import tensorflow as tf

from keras import backend as K
from keras import layers
from keras.backend import set_session

from keras.layers import *
from keras.models import Model

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import *

import warnings
warnings.simplefilter('ignore')

## Tensorflow-v2 の場合。

### TensorFlow 2 で 1.x のコードを走らせるには
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#tf.debugging.set_log_device_placement(True)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

# Optuna 1

In [3]:
def load_data():
    path_dir = "../New-RGB/"

    filelist = pd.read_csv("./new_Mandible_check.csv", encoding="SHIFT-JIS")
    #filelist.columns = ["num","Filename", "Species", "Japanese"]
    japanese_group = filelist.groupby(["Japanese"])
    FILELIST = filelist.sort_values("Filename")
    japanese_file = list(FILELIST.Japanese)
    
    num_list = []

    for i,_ in filelist.iterrows():
            num_list.append(i)

    X = []
    y = []
    y_jap = []
    y_species = []
    y_food = []
    filelise_index = []
    for i in num_list:
        PATH = os.path.join(*[path_dir, filelist.Filename[i]])
        if os.path.isfile(PATH) == True:
            img = cv2.imread(PATH)
            #img = cv2.resize(img, (128,128))
            img = img /255
            X.append(img)
            y.append(filelist.Family_num[i])
            y_jap.append(filelist.Family[i])
            y_species.append(filelist.Japanese[i])
            y_food.append(filelist.Food[i])
        else:
            print(PATH)
        
    X = np.array(X)
    y = np.array(y)
    
    return X,y


    """
    Composition of Data.
    
    data ━━━┳━━> train 67 % ┳━━ train  75 %
            ┃               ┗━━   val  25 %
            ┃
            ┗-- test  33 %
    
    test data ... data for check accuracy, loss et al
    
    
    """



def make_train_test(X,y,seed = 223):

    X_test =[]
    y_test =[]
    X_train = []
    y_train = []
    group_train = []
        
    filelist = pd.read_csv("./new_Mandible_check.csv", encoding="SHIFT-JIS")
    #filelist.columns = ["num","Filename", "Species", "Japanese"]
    for i in range(7):
        Group = filelist[y == i].Tag
        yy = y[y == i]
        #yyy = y_tag[y ==i]
        XX = X[y==i]
        gss = GroupShuffleSplit(n_splits=2, train_size=.67, random_state=seed)
        for train_idx, test_idx in gss.split(XX, yy, Group):
            continue
        #print(yyy[test_idx])
        X_test.append(XX[test_idx])
        X_train.append(XX[train_idx])
        y_test.append(yy[test_idx])
        y_train.append(yy[train_idx])
        group_train.append(np.array(Group)[train_idx])

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    group_train = np.array(group_train)

    for j in range(X_train.shape[0]):
        if j == 0 :
            tmp = X_train[j]
            tmp2 = y_train[j]
            tmp3 = group_train[j]
        else:
            tmp = np.concatenate([tmp, X_train[j]])
            tmp2 = np.concatenate([tmp2,y_train[j]])
            tmp3 = np.concatenate([tmp3,group_train[j]])
    X_train = tmp
    y_train = tmp2
    group_train = tmp3

    for j in range(X_test.shape[0]):
        if j == 0 :
            tmp = X_test[j]
            tmp2 = y_test[j]
        else:
            tmp = np.concatenate([tmp, X_test[j]])
            tmp2 = np.concatenate([tmp2,y_test[j]])
    X_test = tmp
    y_test = tmp2
    print("Train size :{}, Test size:{}".format(np.array(X_train).shape,np.array(X_test).shape))
    Y_train = to_categorical(y_train)
    Y_test = to_categorical(y_test)

    return X_train, y_train,Y_train, X_test, y_test ,Y_test, group_train

seed = 42

def make_val_train(X_train,y_train,group_train, seed = seed):
    X_val =[]
    y_val =[]
    X_train_2 = []
    y_train_2 = []
    for i in range(7):
        Group = group_train[y_train == i]
        yy = y_train[y_train == i]
        #yyy = y_tag[y ==i]
        XX = X_train[y_train==i]
        gss = GroupShuffleSplit(n_splits=2, train_size=.75, random_state=seed)
        for train_idx, test_idx in gss.split(XX, yy, Group):
            continue
        #print(yyy[test_idx])
        X_val.append(XX[test_idx])
        X_train_2.append(XX[train_idx])
        y_val.append(yy[test_idx])
        y_train_2.append(yy[train_idx])
        
    X_train_2 = np.array(X_train_2)
    X_val = np.array(X_val)
    
    for j in range(X_train_2.shape[0]):
            if j == 0 :
                tmp = X_train_2[j]
                tmp2 = y_train_2[j]
            else:
                tmp = np.concatenate([tmp, X_train_2[j]])
                tmp2 = np.concatenate([tmp2,y_train_2[j]])
    X_train_2 = tmp
    y_train_2 = tmp2
    
    for j in range(X_val.shape[0]):
        if j == 0 :
            tmp =X_val[j]
            tmp2 = y_val[j]
        else:
            tmp = np.concatenate([tmp, X_val[j]])
            tmp2 = np.concatenate([tmp2,y_val[j]])
    X_val = tmp
    y_val = tmp2

    Y_train_2 = to_categorical(y_train_2)
    Y_val = to_categorical(y_val)
    print("Train2 size :{}, val size:{}".format(np.array(X_train_2).shape,np.array(X_val).shape))
    
    return X_train_2, y_train_2,Y_train_2, X_val, y_val ,Y_val



In [4]:
def create_model(num_layer, activation, num_filters, latent_dim = 3):
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean =0., stddev =1.)
        return z_mean + K.exp(z_log_var) * epsilon
    
    def vae_loss(inputs, z_decoded):
        x = K.flatten(inputs)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)
   
    inputs = Input((128,128,3))

    x0 = Lambda(lambda x: x[ :,:,:,0:1], output_shape=(128,128,1),name = 'R_extract')(inputs)
    x1 = Lambda(lambda x: x[ :,:,:,1:2], output_shape=(128,128,1),name = 'G_extract')(inputs)
    x2 = Lambda(lambda x: x[:,:,:,2:], output_shape=(128,128,1),name = 'B_extract')(inputs)

    x0 = Convolution2D(filters=num_filters[0], kernel_size=(3,3), padding="same", activation=activation,
                      name = 'R_input')(x0)
    #print(K.int_shape(x0))
    for i in range(1,num_layer):
        x0 = MaxPooling2D((2,2), padding='same', name = 'R_MaxPool_{}'.format(i))(x0)
        x0 = Convolution2D(filters=num_filters[i], kernel_size=(3,3), padding="same", activation=activation,
                          name = 'R_conv_{}'.format(i))(x0)
    x0 = layers.Flatten(name = 'R_flatten')(x0)

    x1 = Convolution2D(filters=num_filters[0], kernel_size=(3,3), padding="same", activation=activation,
                      name = 'G_input')(x1)
    for i in range(1,num_layer):
        x1 = MaxPooling2D((2,2), padding='same',name = 'G_MaxPool_{}'.format(i))(x1)
        x1 = Convolution2D(filters=num_filters[i], kernel_size=(3,3), padding="same", activation=activation
                          ,name = 'G_conv_{}'.format(i))(x1)
    x1 = layers.Flatten(name = 'G_flatten')(x1)

    x2 = Convolution2D(filters=num_filters[0], kernel_size=(3,3), padding="same", activation=activation,
                      name = 'B_input')(x2)
    for i in range(1,num_layer):
        x2 = MaxPooling2D((2,2), padding='same',name = 'B_MaxPool_{}'.format(i))(x2)
        x2 = Convolution2D(filters=num_filters[i], kernel_size=(3,3), padding="same", activation=activation
                          ,name = 'B_conv_{}'.format(i))(x2)
    x2 = layers.Flatten(name = 'B_flatten')(x2)

    x = keras.layers.Concatenate(name = 'Concatenate')([x0, x1, x2])
    shape_before_flattening = K.int_shape(x)


    ## Intermediate Layer Part
    z_mean = layers.Dense(latent_dim, name ='z_mean')(x) #latent Spaceに圧縮
    z_log_var = layers.Dense(latent_dim, name ='z_log_var')(x) #z_sigma に対応している
    z = layers.Lambda(sampling, name = 'sampling')([z_mean, z_log_var])

    encoder = Model(inputs,z)

    decoder_input = layers.Input(K.int_shape(z)[1:])
    x = layers.Dense(np.prod(shape_before_flattening[1:]),activation=activation)(decoder_input)

    Shape = int(128 /(2 ** (num_layer - 1)))
    flatten_shape = int(Shape * Shape * num_filters[-1])

    ## 実質的な Convolution
    x0 = Lambda(lambda x: x[:, :flatten_shape], output_shape=(Shape,Shape,num_filters[-1]))(x)
    x1 = Lambda(lambda x: x[:,  flatten_shape:2 * flatten_shape], output_shape=(Shape,Shape,num_filters[-1]))(x)
    x2 = Lambda(lambda x: x[:, 2*flatten_shape:], output_shape=(Shape,Shape,num_filters[-1]))(x)

    x0 =layers.Reshape((Shape, Shape, num_filters[-1]))(x0)
    x1 =layers.Reshape((Shape, Shape, num_filters[-1]))(x1)
    x2 =layers.Reshape((Shape, Shape, num_filters[-1]))(x2)

    for i in reversed(range(num_layer - 1)):
            x0 = Convolution2D(filters=num_filters[i], kernel_size=(3,3),padding="same", activation=activation)(x0)
            x0 = UpSampling2D((2,2))(x0)
    x0 = Convolution2D(1,(3,3), padding='same', activation='sigmoid')(x0)

    for i in reversed(range(num_layer - 1)):
            x1 = Convolution2D(filters=num_filters[i], kernel_size=(3,3),padding="same", activation=activation)(x1)
            x1 = UpSampling2D((2,2))(x1)
    x1 = Convolution2D(1,(3,3), padding='same', activation='sigmoid')(x1)

    for i in reversed(range(num_layer - 1)):
            x2 = Convolution2D(filters=num_filters[i], kernel_size=(3,3),padding="same", activation=activation)(x2)
            x2 = UpSampling2D((2,2))(x2)
    x2 = Convolution2D(1,(3,3), padding='same', activation='sigmoid')(x2)

    decoder_output = layers.Concatenate()([x0, x1, x2])
    decoder = Model(decoder_input, decoder_output)
    z_decoded = decoder(z)

    classification_output = layers.Dense(7 ,activation='softmax',name = 'Family_Classifier')(z)

    model = Model(inputs, [classification_output, z_decoded])

    Classifier = Model(inputs,classification_output, name ='Classifier')
    
    return model, z_mean, z_log_var, encoder, decoder, Classifier



def objective(trial):
    
    alpha = 0.5
    latent_dim = 3

    #セッションのクリア
    tf.keras.backend.clear_session()
    
    def vae_loss(inputs, z_decoded):
        x = K.flatten(inputs)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)
    
    checkpoint = keras.callbacks.ModelCheckpoint("./WEIGHTS/optuna_tmp_33.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=0)

    earlystop = keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                              min_delta = 0, 
                              patience = 10,
                              verbose = 0)

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                  factor = 0.5,
                                  patience = 10,
                                  verbose = 0)

    callbacks = [earlystop, checkpoint, reduce_lr]
    

    #最適化するパラメータの設定
    #畳込み層の数
    num_layer = trial.suggest_int("num_layer", 1,5)

    #latent_dim = trial.suggest_int("latent_dim", 2,5)

    #各畳込み層のフィルタ数
    num_filters = [int(trial.suggest_discrete_uniform("num_filter_"+str(i), 16, 128, 16)) for i in range(num_layer)]

    #活性化関数
    activation = trial.suggest_categorical("activation", ["relu", "sigmoid", "tanh"])

    #optimizer
    optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])

    model, z_mean, z_log_var, encoder, decoder, Classifier = create_model(num_layer, activation, num_filters,latent_dim)
    model.compile(optimizer=optimizer, 
                  loss = ['categorical_crossentropy',vae_loss], 
                  loss_weights = [alpha, 1-alpha],
                  metrics=['accuracy'])
    
    X_train, y_train,Y_train, X_test, y_test , Y_test, group_train =  make_train_test(X,y)
    
    X_train_2, y_train_2,Y_train_2, X_val, y_val ,Y_val = make_val_train(X_train,y_train,group_train,seed = 42)
    
    history = model.fit(X_train_2,
                        [Y_train_2, X_train_2],
                        epochs=100,
                        batch_size=10,
                        validation_data=(X_val,[Y_val,X_val]),
                        callbacks = callbacks,
                        verbose = 2)
    os.makedirs('./WEIGHTS',exist_ok=True)
    date = '20220202'
    os.makedirs('./WEIGHTS/{}'.format(date), exist_ok=True)
    
    encoder.save_weights('./WEIGHTS/{0}/encoder_{0}_alpha{1}_index{2}_seed{3}.h5'.format(date,alpha, index,seed))
    decoder.save_weights('./WEIGHTS/{0}/decoder_{0}_alpha{1}_index{2}_seed{3}.h5'.format(date,alpha, index,seed))
    Classifier.save_weights('./WEIGHTS/{0}/Classifier_{0}_alpha{1}_index{2}_seed{3}.h5'.format(date,alpha, index,seed))
    model.save_weights('./WEIGHTS/{0}/model_{0}_alpha{1}_index{2}_seed{3}.h5'.format(date,alpha, index,seed))
    
    ## 実際に Lossなどを計算する。
    test_loss, test_reconst_loss, test_class_loss, test_reconst_acc, test_class_acc = model.evaluate(X_test,[Y_test, X_test])
    print(test_loss, test_reconst_loss, test_class_loss, test_reconst_acc, test_class_acc)
    # clear the data to relaese the memory
    del  X_train, y_train,Y_train, X_test, y_test , Y_test, X_train_2, y_train_2, Y_train_2, X_val, y_val, Y_val
    import re 
    pattern = '.*?(\d+).*'
    s = str(history.history.keys())
    number = int(re.findall(r'\d+', s)[0])

    return history.history["val_loss"][-1]

In [5]:
X, y = load_data()


In [20]:
X_train, y_train,Y_train, X_test, y_test , Y_test, group_train =  make_train_test(X,y)

/tmp/ipykernel_89909/1736647027.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train)
/tmp/ipykernel_89909/1736647027.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test)
/tmp/ipykernel_89909/1736647027.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  group_train = np.array(gr

Train size :(194, 128, 128, 3), Test size:(100, 128, 128, 3)


In [21]:
X_train_2, y_train_2,Y_train_2, X_val, y_val ,Y_val = make_val_train(X_train,y_train,group_train, seed = 42)

Train2 size :(144, 128, 128, 3), val size:(50, 128, 128, 3)


/tmp/ipykernel_89909/1736647027.py:132: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_2 = np.array(X_train_2)
/tmp/ipykernel_89909/1736647027.py:133: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_val = np.array(X_val)


In [ ]:
try:
  # Specify an invalid GPU device
    for index in range(100):
        tf.config.experimental.set_virtual_device_configuration(gpus[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        print("{} th trial".format(index))
        study = optuna.create_study()
        study.optimize(objective, n_trials=100)
        df = study.trials_dataframe()
        df.to_csv("./Results/result_20220202/VAE_optuna_tuning_result_20210202_seed_42_trial{}.csv".format(index))
except RuntimeError as e:
    print(e)

[I 2022-02-03 15:22:59,303] A new study created in memory with name: no-name-c4972763-5ae7-49ec-8e9c-cea8ad1b4fb1


0 th trial
Train size :(194, 128, 128, 3), Test size:(100, 128, 128, 3)
Train2 size :(144, 128, 128, 3), val size:(50, 128, 128, 3)
Train on 144 samples, validate on 50 samples


2022-02-03 15:23:01.125983: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-03 15:23:03.513539: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4094 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:1b:00.0, compute capability: 8.6
2022-02-03 15:23:03.515609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 4096 MB memory:  -> device: 1, name: GeForce RTX 3090, pci bus id: 0000:20:00.0, compute capability: 8.6
2022-02-03 15:23:03.516768: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

Epoch 1/100


2022-02-03 15:23:05.765867: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2022-02-03 15:23:07.964377: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


...

Epoch 1/100


2022-02-03 15:23:24.447225: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.70GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


144/144 - 2s - loss: nan - Family_Classifier_loss: nan - model_1_loss: nan - Family_Classifier_acc: 0.5625 - model_1_acc: 0.8612 - val_loss: nan - val_Family_Classifier_loss: nan - val_model_1_loss: nan - val_Family_Classifier_acc: 0.6000 - val_model_1_acc: 0.9456 - lr: 0.0010 - 2s/epoch - 15ms/sample
Epoch 2/100
144/144 - 1s - loss: nan - Family_Classifier_loss: nan - model_1_loss: nan - Family_Classifier_acc: 0.5903 - model_1_acc: 0.9465 - val_loss: nan - val_Family_Classifier_loss: nan - val_model_1_loss: nan - val_Family_Classifier_acc: 0.6000 - val_model_1_acc: 0.9456 - lr: 0.0010 - 701ms/epoch - 5ms/sample
Epoch 3/100
144/144 - 1s - loss: nan - Family_Classifier_loss: nan - model_1_loss: nan - Family_Classifier_acc: 0.5903 - model_1_acc: 0.9465 - val_loss: nan - val_Family_Classifier_loss: nan - val_model_1_loss: nan - val_Family_Classifier_acc: 0.6000 - val_model_1_acc: 0.9456 - lr: 0.0010 - 741ms/epoch - 5ms/sample
Epoch 4/100
144/144 - 1s - loss: nan - Family_Classifier_loss: 

2022-02-03 15:23:34.834079: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-03 15:23:34.834350: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-03 15:23:34.834500: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-03 15:23:34.834915: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Alloc

nan nan nan 0.59 0.9469318
Train size :(194, 128, 128, 3), Test size:(100, 128, 128, 3)


In [ ]:
histo